# EE Retrieval

In [399]:
#### For this to work you need to replace w/ your own service account and credentials
#### You may also need to install the earthengine-api
#### ! pip install earthengine-api --upgrade

# Service Account, eg "[...]iam.gserviceaccount.com"
service_account = ''
# Path to the service account key
json_credentials = ''

# Only need to initialize once in awhile
auth_intialize = False

In [400]:
import ee
import numpy as np
import pandas as pd
import requests
from PIL import Image

In [401]:
if auth_intialize:
    SCOPES = [
        'https://www.googleapis.com/auth/earthengine',
        'https://www.googleapis.com/auth/drive'
    ]
    ee.Authenticate(scopes=SCOPES)
    credentials = ee.ServiceAccountCredentials(service_account, json_credentials)
    ee.Initialize(credentials)

## Check collections for images

In [402]:
# Example coordinates
lat, lon = 37.868710, -122.274720
point = ee.Geometry.Point([lon, lat])

# Create a buffer around the point
buffer_distance = 100
roi = point.buffer(buffer_distance)

# Define the date range of interest
start_date = '2024-01-01'
end_date = '2024-04-02'

# List of image collections to check
collections = [
    'COPERNICUS/S2_SR_HARMONIZED',
    'COPERNICUS/S1_GRD',
    'LANDSAT/LC08/C01/T1_SR',
    'LANDSAT/LE07/C01/T1_SR',
    'LANDSAT/LT05/C01/T1_SR',
    'MODIS/006/MOD09GQ',
    'MODIS/006/MOD13Q1',
    'NOAA/GOES/16/MCMIPF',
    'COPERNICUS/S5P/OFFL/L3_NO2'
]

# Check each collection for images in the specified date range
available_collections = []

for collection_id in collections:
    collection = ee.ImageCollection(collection_id).filterDate(start_date, end_date).filterBounds(roi)
    count = collection.size().getInfo()
    if count > 0:
        available_collections.append(collection_id)
        print(f"{collection_id} has {count} images available.")
    else:
        print(f"{collection_id} has no images available for the specified date range: {start_date} to {end_date}")

if not available_collections:
    print("No collections have images available for the specified date range and region.")

print("Available collections with images:", available_collections)

COPERNICUS/S2_SR_HARMONIZED has 34 images available.
COPERNICUS/S1_GRD has 23 images available.
LANDSAT/LC08/C01/T1_SR has no images available for the specified date range: 2024-01-01 to 2024-04-02
LANDSAT/LE07/C01/T1_SR has no images available for the specified date range: 2024-01-01 to 2024-04-02
LANDSAT/LT05/C01/T1_SR has no images available for the specified date range: 2024-01-01 to 2024-04-02
MODIS/006/MOD09GQ has no images available for the specified date range: 2024-01-01 to 2024-04-02
MODIS/006/MOD13Q1 has no images available for the specified date range: 2024-01-01 to 2024-04-02
NOAA/GOES/16/MCMIPF has 13203 images available.
COPERNICUS/S5P/OFFL/L3_NO2 has 1286 images available.
Available collections with images: ['COPERNICUS/S2_SR_HARMONIZED', 'COPERNICUS/S1_GRD', 'NOAA/GOES/16/MCMIPF', 'COPERNICUS/S5P/OFFL/L3_NO2']


## Retrieve images

In [391]:
def process_sentinel2(date1, date2, lon, lat, export_desc, buffer=2560, cloud_percentage=20, export=False):    
    # Create point geometry
    coordinates = ee.Geometry.Point([lon, lat])
    
    # Define region of interest
    region = coordinates.buffer(buffer)

    # Load Sentinel-2 collection
    # Sentinel-2 has a max resolution of 10 m/pixel
    # Also has a 5-day revisit time
    # https://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-2
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                    .filterBounds(region) \
                    .filterDate(date1, date2) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))

    # Create a mosaic of filtered images
    # https://developers.google.com/earth-engine/guides/ic_composite_mosaic
    mosaic = sentinel2.median()
 
    # Process and download true-color and NDVI images

    # true-color images
    # cast tc bands to the same data type
    image_uint16 = mosaic.select(['B4', 'B3', 'B2']).uint16()
    tc_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 500,
        'max': 4000,
        'gamma': 1.4  # adjust for contrast
    }
    true_color_img = image_uint16.visualize(**tc_params)    

    # ndvi images
    ndvi_raw_img = mosaic.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # ndvi two-tone color
    ndvi_color_params = {
        'min': -1,
        'max': 1,
        'palette': ['darkblue', 'white']
    }
    # ndvi_color_img = ndvi_raw_img.visualize(**ndvi_color_params)    
    
    # ndvi grayscale
    ndvi_bw_params = {
        'min': -1,
        'max': 1,
        'palette': ['black', 'white']
    }
    ndvi_bw_img = ndvi_raw_img.visualize(**ndvi_bw_params)

    image_dests = []
    image_dests.append(download_cropped_image(true_color_img, region, export_desc + "_tc"))
    # image_dests.append(download_image(ndvi_color_img, region, export_desc + "_ndvi_color"))
    image_dests.append(download_cropped_image(ndvi_bw_img, region, export_desc + "_ndvi_bw"))

    # Export image to Google Drive
    if export:
        raise ValueError("Export to drive should be configured separately")
        # NOTE: comment out the error above if export to drive has been configured
        export_task = ee.batch.Export.image.toDrive(
            image=image_uint16.visualize(**visualization),
            description=export_desc,
            folder='EarthEngineExports',
            scale=10,  # in meters
            region=region.bounds().getInfo()['coordinates'],
            fileFormat='GeoTIFF',
            crs='EPSG:4326',
            maxPixels=1e9
        )

        export_task.start()
    
    return image_dests

## Download and crop images to standard size

In [392]:
def download_cropped_image(image, region, export_desc=None, scale=10, crop_size=512):
    # Download image
    url = image.getDownloadURL({
        'scale': scale,
        'region': region.bounds().getInfo()['coordinates'],
        'format': 'GeoTIFF',
        'crs': 'EPSG:4326'
    })
    response = requests.get(url, stream=True)

    # Write to disk and crop
    if response.status_code == 200:
        export_raw_fpath = f"./{export_desc}.tif"
        # export raw image
        with open(export_raw_fpath, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        # crop image to center square
        final_image_size = crop_tif_as_center_square(export_raw_fpath)
        print(f"Image download: {export_raw_fpath} {final_image_size}")
        return export_raw_fpath
    else:
        print(f"Failed to download image. HTTP status code: {response.status_code}")
        return None

In [393]:
def crop_center(img, crop_width, crop_height):
    img_width, img_height = img.size
    return img.crop((
        (img_width - crop_width) // 2,
        (img_height - crop_height) // 2,
        (img_width + crop_width) // 2,
        (img_height + crop_height) // 2
    ))

def crop_tif_as_center_square(fpath, crop_size=512):
    img = Image.open(fpath)
    cropped_img = crop_center(img, crop_size, crop_size)
    cropped_img.save(fpath)
    return cropped_img.size

In [364]:
def example_test(fname, true_color):
    # Define date range
    date1 = '2024-06-01'
    date2 = '2024-06-30'

    # Define coordinates

    # Test 1
    # Golden Gate Park, SF
    lon1 = -122.476944
    lat1 = 37.769722
    
    # Test 2
    # Berkeley, CA
    lon2 = -122.274720
    lat2 = 37.868710

    # Define buffer area
    # (m around lat/lon)
    buffer = 2560 

    # Define cloud percentage
    cloud_percentage = 20

    # Specify export description
    # related to export filename
    export_desc = fname

    process_sentinel2(date1, date2, lon1, lat1, export_desc + "_t1", buffer, cloud_percentage)
    process_sentinel2(date1, date2, lon2, lat2, export_desc + "_t2", buffer, cloud_percentage)


example_test('0724__test_v16', True)

Image download: ./0724__test_v16_t1_tc.tif (512, 512)
Image download: ./0724__test_v16_t1_ndvi_bw.tif (512, 512)
Image download: ./0724__test_v16_t2_tc.tif (512, 512)
Image download: ./0724__test_v16_t2_ndvi_bw.tif (512, 512)


## Load wildfires dataset

In [365]:
wildfires_df = pd.read_csv("./California_Fire_Incidents.csv")
wildfires_df.head(2)

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN


In [366]:
wildfires_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AcresBurned           1633 non-null   float64
 1   Active                1636 non-null   bool   
 2   AdminUnit             1636 non-null   object 
 3   AirTankers            28 non-null     float64
 4   ArchiveYear           1636 non-null   int64  
 5   CalFireIncident       1636 non-null   bool   
 6   CanonicalUrl          1636 non-null   object 
 7   ConditionStatement    284 non-null    object 
 8   ControlStatement      105 non-null    object 
 9   Counties              1636 non-null   object 
 10  CountyIds             1636 non-null   object 
 11  CrewsInvolved         171 non-null    float64
 12  Dozers                123 non-null    float64
 13  Engines               191 non-null    float64
 14  Extinguished          1577 non-null   object 
 15  Fatalities           

In [367]:
wildfires_sm_df = wildfires_df[['Longitude', 'Latitude', 'Started']].copy()
wildfires_sm_df.tail()

,Longitude,Latitude,Started
1631,-117.499619,33.827979,2019-10-10T12:08:00Z
1632,-121.000556,39.409722,2019-06-28T15:03:04Z
1633,-121.729691,38.734634,2019-11-25T12:02:02Z
1634,-117.403719,33.351145,2019-10-22T19:20:44Z
1635,-116.058980,33.451480,2019-10-14T15:32:20Z


In [368]:
# Convert started col to dt
wildfires_sm_df['Started'] = pd.to_datetime(wildfires_sm_df['Started'], format='ISO8601')

In [369]:
# Get dt one day and one day + one week before fire started
# This is to account for satellite revisit time and to avoid the date of the fire
wildfires_sm_df['Day_Before_DT'] = wildfires_sm_df['Started'] - pd.Timedelta(days=1)
wildfires_sm_df['Week_Before_DT'] = wildfires_sm_df['Day_Before_DT'] - pd.Timedelta(weeks=1)
wildfires_sm_df['Day_Before'] = wildfires_sm_df['Day_Before_DT'].dt.date.astype(str)
wildfires_sm_df['Week_Before'] = wildfires_sm_df['Week_Before_DT'].dt.date.astype(str)

In [370]:
wildfires_sm_df.head()

,Longitude,Latitude,Started,Day_Before_DT,Week_Before_DT,Day_Before,Week_Before
0,-120.086000,37.857000,2013-08-17 15:25:00+00:00,2013-08-16 15:25:00+00:00,2013-08-09 15:25:00+00:00,2013-08-16,2013-08-09
1,-118.423176,34.585595,2013-05-30 15:28:00+00:00,2013-05-29 15:28:00+00:00,2013-05-22 15:28:00+00:00,2013-05-29,2013-05-22
2,-116.728850,33.709500,2013-07-15 13:43:00+00:00,2013-07-14 13:43:00+00:00,2013-07-07 13:43:00+00:00,2013-07-14,2013-07-07
3,-120.650000,39.120000,2013-08-10 16:30:00+00:00,2013-08-09 16:30:00+00:00,2013-08-02 16:30:00+00:00,2013-08-09,2013-08-02
4,0.000000,0.000000,2013-05-02 07:01:00+00:00,2013-05-01 07:01:00+00:00,2013-04-24 07:01:00+00:00,2013-05-01,2013-04-24


In [371]:
wildfires_test_set = wildfires_sm_df.tail(10)
wildfires_test_set

,Longitude,Latitude,Started,Day_Before_DT,Week_Before_DT,Day_Before,Week_Before
1626,-120.671310,38.332083,2019-09-25 13:13:41+00:00,2019-09-24 13:13:41+00:00,2019-09-17 13:13:41+00:00,2019-09-24,2019-09-17
1627,-116.631106,33.496633,2019-09-10 10:43:58+00:00,2019-09-09 10:43:58+00:00,2019-09-02 10:43:58+00:00,2019-09-09,2019-09-02
1628,-121.579738,39.423833,2019-07-23 14:41:00+00:00,2019-07-22 14:41:00+00:00,2019-07-15 14:41:00+00:00,2019-07-22,2019-07-15
1629,-122.401570,41.946220,2019-06-16 20:33:00+00:00,2019-06-15 20:33:00+00:00,2019-06-08 20:33:00+00:00,2019-06-15,2019-06-08
1630,-121.957000,39.839580,2019-04-30 12:20:00+00:00,2019-04-29 12:20:00+00:00,2019-04-22 12:20:00+00:00,2019-04-29,2019-04-22
1631,-117.499619,33.827979,2019-10-10 12:08:00+00:00,2019-10-09 12:08:00+00:00,2019-10-02 12:08:00+00:00,2019-10-09,2019-10-02
1632,-121.000556,39.409722,2019-06-28 15:03:04+00:00,2019-06-27 15:03:04+00:00,2019-06-20 15:03:04+00:00,2019-06-27,2019-06-20
1633,-121.729691,38.734634,2019-11-25 12:02:02+00:00,2019-11-24 12:02:02+00:00,2019-11-17 12:02:02+00:00,2019-11-24,2019-11-17
1634,-117.403719,33.351145,2019-10-22 19:20:44+00:00,2019-10-21 19:20:44+00:00,2019-10-14 19:20:44+00:00,2019-10-21,2019-10-14
1635,-116.058980,33.451480,2019-10-14 15:32:20+00:00,2019-10-13 15:32:20+00:00,2019-10-06 15:32:20+00:00,2019-10-13,2019-10-06


In [372]:
def retrieve_set(row):
    try:
        process_sentinel2(date1=row['Week_Before'], date2=row['Day_Before'], 
                          lon=row['Longitude'], lat=row['Latitude'], 
                          export_desc=f"{row['Week_Before']}-{row['Day_Before']}-{row['Longitude']}-{row['Latitude']}")
    except Exception as e:
        print(e)

In [373]:
wildfires_test_set.apply(lambda row: retrieve_set(row), axis=1)

Image download: ./2019-09-17-2019-09-24--120.67131-38.332083_tc.tif (512, 512)
Image download: ./2019-09-17-2019-09-24--120.67131-38.332083_ndvi_bw.tif (512, 512)
Image download: ./2019-09-02-2019-09-09--116.631106-33.496633_tc.tif (512, 512)
Image download: ./2019-09-02-2019-09-09--116.631106-33.496633_ndvi_bw.tif (512, 512)
Image download: ./2019-07-15-2019-07-22--121.579738-39.423833_tc.tif (512, 512)
Image download: ./2019-07-15-2019-07-22--121.579738-39.423833_ndvi_bw.tif (512, 512)
Image download: ./2019-06-08-2019-06-15--122.40157-41.94622_tc.tif (512, 512)
Image download: ./2019-06-08-2019-06-15--122.40157-41.94622_ndvi_bw.tif (512, 512)
Image download: ./2019-04-22-2019-04-29--121.957-39.83958_tc.tif (512, 512)
Image download: ./2019-04-22-2019-04-29--121.957-39.83958_ndvi_bw.tif (512, 512)
Image download: ./2019-10-02-2019-10-09--117.499619-33.827979_tc.tif (512, 512)
Image download: ./2019-10-02-2019-10-09--117.499619-33.827979_ndvi_bw.tif (512, 512)
Image download: ./2019-0

1626    None
1627    None
1628    None
1629    None
1630    None
1631    None
1632    None
1633    None
1634    None
1635    None
dtype: object